In [1]:
# Import modules
import numpy as np
import librosa.display, os
import matplotlib.pyplot as plt
import keras.utils as image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_curve
import numpy as np
from keras.layers import GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications.mobilenet import MobileNet
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications.mobilenet import MobileNet
# Import modules
import tensorflow as tf
import gc

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense

import numpy as np
import librosa.display, os

# import keras.utils as image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam


In [ ]:
X = np.load('../SavedFeatures/X_mel_spec.npy')
y = np.load('../SavedFeatures/y_mel_spec.npy')

X_train, X_val, y_train, y_val= train_test_split(X, y, test_size = 0.2, random_state = 42)

y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

del X, y  
gc.collect()

In [ ]:
def model_mobilenet(input_shape, num_classes):
    base_model = MobileNet(input_shape=input_shape, weights=None, include_top=False)

    x = GlobalAveragePooling2D()(base_model.output)
    # x = BatchNormalization()(x)  
    x = Dropout(0.5)(x)  
    
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)  
    x = Dropout(0.4)(x)  
    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)  
    
    # x = Dropout(0.2)(x) 
    output = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=output)

In [18]:
# Release GPU
tf.keras.backend.clear_session()

In [19]:
mbnv3 = model_mobilenet(input_shape=(N_MELS, 109, 1), num_classes=2)
mbnv3.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 109, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 64, 55, 32)     │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 64, 55, 32)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 64, 55, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 64, 55, 32)     │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 64, 55, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 64, 55, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 64, 55, 64)     │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 64, 55, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 64, 55, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 65, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 32, 27, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 32, 27, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 32, 27, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 32, 27, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 32, 27, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 32, 27, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 32, 27, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 32, 27, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 32, 27, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 32, 27, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3_bn                    │ (None, 32, 27, 128)    │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 3,524,866 (13.45 MB)

 Trainable params: 3,502,466 (13.36 MB)

 Non-trainable params: 22,400 (87.50 KB)

In [20]:
mbnv3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
# Train the Model
hist = mbnv3.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 19s 17ms/step - accuracy: 0.5638 - loss: 0.8472 - val_accuracy: 0.5987 - val_loss: 1.8477
Epoch 2/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.6797 - loss: 0.6704 - val_accuracy: 0.7000 - val_loss: 0.6467
Epoch 3/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7766 - loss: 0.5241 - val_accuracy: 0.7990 - val_loss: 0.4631
Epoch 4/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8326 - loss: 0.4066 - val_accuracy: 0.8685 - val_loss: 0.3242
Epoch 5/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8628 - loss: 0.3542 - val_accuracy: 0.8237 - val_loss: 0.4395
Epoch 6/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8741 - loss: 0.3190 - val_accuracy: 0.8347 - val_loss: 0.4032
Epoch 7/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8870 - loss: 0.2946 - val_accuracy: 0.8999 - val_loss: 0.2720
Epoch 8/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8951 - loss: 0.2706 - val_accuracy: 

In [21]:
test_loss, test_accuracy = mbnv3.evaluate(X_val, y_val, verbose=0)
y_pred = mbnv3.predict(X_val)
y_pred_classes = y_pred.argmax(axis=1) 
y_true_classes = y_val.argmax(axis=1) 


2025-03-08 17:14:17.472063: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_707_0', 8 bytes spill stores, 8 bytes spill loads

2025-03-08 17:14:17.651409: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_707_0', 100 bytes spill stores, 100 bytes spill loads

2025-03-08 17:14:17.957414: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_707', 184 bytes spill stores, 184 bytes spill loads

2025-03-08 17:14:18.850888: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_707_0', 8 bytes spill stores, 8 bytes spill loads

2025-03-08 17:14:19.057001: I external/local_xla/xla/strea

80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step


In [ ]:
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
print("===================MobileNet - MS===================")
print(f"TAccuracy: {test_accuracy:.5f}")
print(f"F1-Score: {f1:.5f}")
eers = []

# Tính EER 
for i in range(y_pred.shape[1]):  
    
    y_true_binary = y_val[:, i]
    y_pred_prob = y_pred[:, i]

    
    fpr, tpr, thresholds = roc_curve(y_true_binary, y_pred_prob)
    fnr = 1 - tpr
    
    eer_threshold = thresholds[np.nanargmin(np.abs(fpr - fnr))]
    eer = fpr[np.nanargmin(np.abs(fpr - fnr))]
    eers.append(eer)
    print(f"Class {i}: EER = {eer:.5f} at threshold {eer_threshold:.5f}")


mean_eer = np.mean(eers)
print(f"EER: {mean_eer:.5f}")


===================MobileNet - MS===================
TAccuracy: 0.91402
F1-Score: 0.91297
Class 0: EER = 0.10274 at threshold 0.85216
Class 1: EER = 0.10295 at threshold 0.14877
EER: 0.10285


In [24]:
# Save model as weights in H5 format
mbnv3.save_weights("MS_mbnv1.weights.h5")

In [ ]:

del mbnv3, X_train, X_val, y_train, y_val 
gc.collect()


23108